<a href="https://colab.research.google.com/github/AbhiDhariwal/Machine-Learning/blob/master/Hindi_Spacy_Working_NER_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec

--2020-03-23 18:00:20--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 416644179 (397M) [binary/octet-stream]
Saving to: ‘wiki.hi.vec’

wiki.hi.vec         100%[===================>] 397.34M  39.8MB/s    in 10s     

2020-03-23 18:00:30 (38.9 MB/s) - ‘wiki.hi.vec’ saved [416644179/416644179]



In [0]:
import spacy
import random # random function for to remove bais in Traning Data
import numpy
# for batch parsing 
from spacy.util import minibatch, compounding

# For evaluateing the model from testing set
from spacy.gold import GoldParse
from spacy.scorer import Scorer


In [0]:
lang = "hi"
vectors_loc = "wiki.hi.vec"
nlp = spacy.blank(lang)    
with open(vectors_loc, "rb") as file_:        
    header = file_.readline()        
    nr_row, nr_dim = header.split()        
    nlp.vocab.reset_vectors(width=int(nr_dim))        
    for line in file_:            
        line = line.rstrip().decode("utf8")            
        pieces = line.rsplit(" ", int(nr_dim))            
        word = pieces[0]            
        vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
        nlp.vocab.set_vector(word, vector)  # add the vectors to the vocab   

In [9]:
text = "भारी बारिश के कारण आज कार्यालय बंद रहेगा"    
doc = nlp(text)    
print(text, doc[0].similarity(doc[3]))

भारी बारिश के कारण आज कार्यालय बंद रहेगा 0.41513643


In [0]:
import pickle

# Part 2

Steps invloved :-
1. Loading JSON File of  Traning and Testing Data
2. Formating JSON file to required format for training

{ Loading the New Testing Set (Using On Unseen Data Set)  to avoid overfitting of the training data }

In [11]:
filename = "train"
output = open(filename + '.pkl', 'rb')
TRAIN_DATA = pickle.load(output)
TRAIN_DATA[:3]

[('अरे सारथी जम्मू कैसा मौसम है', {'entities': [(10, 15, 'U-location')]}),
 ('अभी दिल्ली में कैसा मौसम है',
  {'entities': [(0, 3, 'U-date'), (4, 10, 'U-location')]}),
 ('आज के लिए मौसम रिपोर्ट बताओ', {'entities': [(0, 2, 'U-date')]})]

In [12]:
filename = "val"
output = open(filename + '.pkl', 'rb')
TEST_DATA = pickle.load(output)
TEST_DATA[0:3] 

[('आज मौसम कैसा है', {'entities': [(0, 2, 'U-date')]}),
 ('क्या हरिद्वार में बहुत ठंड है अभी',
  {'entities': [(5, 13, 'U-location'),
    (23, 26, 'U-weather_type'),
    (30, 33, 'U-date')]}),
 ('आज इतनी गर्मी क्यों है',
  {'entities': [(0, 2, 'U-date'), (8, 13, 'U-weather_type')]})]

# Part  3

Steps involved :- 
1. Loading required library
2. Setting up the parameters for traning
3. Traning and saving the best model

In [0]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing 
from spacy.util import minibatch, compounding

# For evaluateing the model from testing set
from spacy.gold import GoldParse
from spacy.scorer import Scorer


In [0]:
def train_spacy(TRAIN_DATA,TEST_DATA,iterations,droprate = 0.5,modelName = "modelTrained"):

    # loading hindi model and using vector from fasttext
    lang = "hi"
    vectors_loc = "wiki.hi.vec"
    modiner = spacy.blank(lang)    
    with open(vectors_loc, "rb") as file_:        
        header = file_.readline()        
        nr_row, nr_dim = header.split()        
        modiner.vocab.reset_vectors(width=int(nr_dim))        
        for line in file_:            
            line = line.rstrip().decode("utf8")            
            pieces = line.rsplit(" ", int(nr_dim))            
            word = pieces[0]            
            vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
            modiner.vocab.set_vector(word, vector)  # add the vectors to the vocab    

#     modiner = spacy.blank('en')  # create blank Language class
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in modiner.pipe_names:
        ner = modiner.create_pipe('ner')
        modiner.add_pipe(ner, last=True)
     
    # setting up f1score
    f1score = 0.0000

    
    # add labels that will be involved in training 
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in modiner.pipe_names if pipe != 'ner']
    with modiner.disable_pipes(*other_pipes):  # only train NER
        optimizer = modiner.begin_training()
        
        # --Iterations Starts--
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            #--Shuffling Traning Data--
            random.shuffle(TRAIN_DATA)
            losses = {}
            
                      
                    
            # batch Traning For better Training and Learning of model
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                modiner.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=droprate,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print(losses)
            
            
            # Evaluating the Current Model Score 
            results = evaluate(modiner, TEST_DATA)
            print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])
            

            
            # loading previous best saved model in start of traning 
            if f1score == 0.00:
                try:                    
                    pnlp = spacy.load(modelName)
                    result = evaluate(pnlp, TEST_DATA) # calling evaluate function 
                    f1score = result["ents_f"]
                except:
                    print("Previous Model not found")
                    
            print("Best Sccore :- ",f1score)
            print("------------------------------------")
            # finding out the best score
            if f1score < results["ents_f"]:
                f1score = results["ents_f"]
                
                # Save our trained Model if the score if grater than best score else no change in previous model
                modiner.to_disk(modelName)
                
    print("-----Best Model is Saved-----")
     


In [0]:
def evaluate(ner_model, examples):
    scorer = Scorer()
     
    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)
        
        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)
            
        # Evaluating the tags    
        gold = GoldParse(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
        
        
    return scorer.scores


In [0]:
def loadNERModel(modelName = "modelTrained"):
    pnlp = spacy.load(modelName)
    return pnlp

In [0]:
def score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function 
    f1score = result["ents_f"]
    precision = result["ents_p"]
    recall = result["ents_r"]
    print("F1 score of Model is :-",f1score)
    print("Precision of Model is :-",precision)
    print("Recall of Model is :-",recall)

In [18]:
# traning the model with 50 iterations


train_spacy(TRAIN_DATA,TEST_DATA, 100,droprate = 0.55, modelName = "hindiNER")

Starting iteration 0
{'ner': 882.2542223148048}
Current Score :- 0.0 Precision  :- 0.0 Recall  :- 0.0
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 1
{'ner': 603.9996254785219}
Current Score :- 79.99999999999999 Precision  :- 85.0 Recall  :- 75.55555555555556
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 2
{'ner': 411.7079789879572}
Current Score :- 74.99999999999999 Precision  :- 76.74418604651163 Recall  :- 73.33333333333333
Best Sccore :-  79.99999999999999
------------------------------------
Starting iteration 3
{'ner': 317.4945251788374}
Current Score :- 94.38202247191012 Precision  :- 95.45454545454545 Recall  :- 93.33333333333333
Best Sccore :-  79.99999999999999
------------------------------------
Starting iteration 4
{'ner': 209.1676732324347}
Current Score :- 94.38202247191012 Precision  :- 95.45454545454545 Recall  :- 93.33333333333333
Best Sccore :-  94.3820224719

In [0]:
# loading the saved model
pnlp = loadNERModel("hindiNER")

In [20]:
# calculating the score of the model
score(pnlp,TEST_DATA)

F1 score of Model is :- 96.62921348314609
Precision of Model is :- 97.72727272727273
Recall of Model is :- 95.55555555555556


In [0]:
from spacy import displacy

In [22]:
testcase = pnlp(TEST_DATA[0][0])
displacy.render(testcase, style='ent', jupyter=True)

# Training Data Result

In [36]:

for i in range(0,len(TRAIN_DATA)):
    testcase = pnlp(TRAIN_DATA[i][0])
    displacy.render(testcase, style='ent', jupyter=True)
#     print("--")

# Testing Data Result

In [35]:
for i in range(0,len(TEST_DATA)):
    try:        
        testcase = pnlp(TEST_DATA[i][0])
        displacy.render(testcase, style='ent', jupyter=True)
#         print("--")
    except:
        a=None

In [30]:
!zip -r /content/hindiNER.zip /content/hindiNER

  adding: content/hindiNER/ (stored 0%)
  adding: content/hindiNER/meta.json (deflated 37%)
  adding: content/hindiNER/tokenizer (deflated 87%)
  adding: content/hindiNER/ner/ (stored 0%)
  adding: content/hindiNER/ner/model (deflated 8%)
  adding: content/hindiNER/ner/moves (deflated 81%)
  adding: content/hindiNER/ner/cfg (deflated 49%)
  adding: content/hindiNER/vocab/ (stored 0%)
  adding: content/hindiNER/vocab/lexemes.bin (deflated 67%)
  adding: content/hindiNER/vocab/key2row (deflated 14%)
  adding: content/hindiNER/vocab/strings.json (deflated 81%)
  adding: content/hindiNER/vocab/vectors (deflated 9%)


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
